In [51]:
from evaluateResults import getExampleRuntimes, getModelTestResults
import pandas as pd
import numpy as np
from pint import UnitRegistry
from typing import Literal

Data setup

In [ ]:
TEST_NUM = 1

with open("../evaluation/titles.json", "r") as f:
    import json 
    exTitles = tuple(json.load(f))

with open(f"../info{TEST_NUM}.json", "r") as f:
    testInfo = json.load(f)
    testTaskIsFast = tuple(testInfo["choices"])

runtimesBaseline = runtimesFast = np.array(getExampleRuntimes("codeFast"))
runtimesSlow = np.array(getExampleRuntimes("codeSlow"))

Table setup

In [ ]:
txtCols = []
timeCols = []
qtyCols = []
boolCols = []
choiceCols = []

In [54]:
colExTitle = "ex.title"
colExFast = "ex.fast.rt (=: bl)"
colExSlow = "log10(ex.slow.rt / bl)"

txtCols.append(colExTitle)
timeCols.append(colExFast)
qtyCols.append(colExSlow)

exampleDf = pd.DataFrame(
    {
        colExTitle: exTitles,
        colExFast: runtimesFast,
        colExSlow: np.log10(runtimesSlow / runtimesBaseline),
    }
)

In [ ]:
colTestTaskIsFast = "test1.task.isFast"

boolCols.append(colTestTaskIsFast)

testDf = pd.DataFrame(
    {colTestTaskIsFast: testTaskIsFast},
)

In [55]:
claudeResults = getModelTestResults(testNum=1, model="claude-sonnet-4")
colTest1ClaudeImproved = "test1.claude4.improved"
colTest1ClaudeRt = "log10(test1.claude4.rt / bl)"

choiceCols.append(colTest1ClaudeImproved)
qtyCols.append(colTest1ClaudeRt)

claudeDf = pd.DataFrame(
    {
        colTest1ClaudeImproved : claudeResults.improvedInfo,
        colTest1ClaudeRt : np.log10(claudeResults.runtimes / runtimesBaseline)
    }
)

Export

In [56]:
ureg = UnitRegistry()

def fmtChoice(x : Literal["y", "n", "~"]) -> str:
    match x:
        case "y": return r"\fullcirc"
        case "n": return r"\emptycirc"
        case "~": return r"\halfcirc"
        case _: raise ValueError


fmt = {
    "txt": lambda x: x,
    "time": lambda t: "{q:.1f~#Lx}".format(q=ureg.Quantity(t, ureg.second)),
    "qty": lambda x: r"\num{" + f"{x:.2f}" + r"}",
    "choice": fmtChoice,
}

fmtMap = {}
fmtMap |= dict.fromkeys(txtCols, fmt["txt"])
fmtMap |= dict.fromkeys(timeCols, fmt["time"])
fmtMap |= dict.fromkeys(qtyCols, fmt["qty"])
fmtMap |= dict.fromkeys(choiceCols, fmt["choice"])

export = pd.concat((exampleDf, claudeDf), axis=1).head()
export.index += 1
print(export.to_latex(
    header=False,
    formatters=fmtMap,
))

\begin{tabular}{llrrlr}
\toprule
\midrule
1 & Ineff. string concat. in loop & \SI[]{201.5}{\micro\second} & \num{1.89} & \halfcirc & \num{0.27} \\
2 & Reserving vector capacity & \SI[]{56.8}{\micro\second} & \num{0.69} & \halfcirc & \num{-0.77} \\
3 & Excessive stream flushing & \SI[]{501.1}{\micro\second} & \num{0.99} & \emptycirc & \num{0.01} \\
4 & Passing/returning vector by value & \SI[]{10.8}{\micro\second} & \num{0.00} & \halfcirc & \num{-0.47} \\
5 & Repeated lookups with std::find & \SI[]{624.8}{\micro\second} & \num{0.17} & \fullcirc & \num{-0.05} \\
\bottomrule
\end{tabular}



In [59]:
export[colExTitle].name

'ex.title'

In [58]:
raise ValueError

ValueError: 

In [64]:
r"\cellcolor{red!25}{%(content)s}" % dict(content=1)

'\\cellcolor{red!25}{1}'